# *MACHINE MANTEINANCE PREDICTOR*

# Purpose

This project aims to apply a regression models to the 4-year history of usage parts, in order to improve the Service Level Agreement perfomance.

In [3]:
!pip install faker

You should consider upgrading via the '/Users/talitacoll/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [92]:
#Import libraries
import string
import random
import pandas as pd
import numpy as np
from time import time
from faker import Faker
import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
import csv
import requests
import json
import io
import warnings
warnings.filterwarnings("ignore")

# Data

As the information is not in the public domain, an exhaustive transformation of the data was carried out, keeping only generic data so as not to lose the dimensionality of the situation.
Otherwise we could not apply the model in a real situation.

**DATA WRANGLING**


*Many DataFrames will be created and each one has their own reason to be*

*Dimension tables:*

pdf = finish goods and features

mdf = modules and features

sdf = spares and features

cdf = customers

*Fact tables:*

ibdf = finish good installed base

Any other fact table are the DataFrame where are working to prepare analysis and test models

In [5]:
#Definy only relevant columns for analysis
cols = ['id','doc_type','transaction_date','order_type','order_date','quantity','unit_cost','extended_cost','item_type']

In [7]:
#The Excel file consist in stock movement
#How data is not public, I didn´t put in a public repository.
df = pd.read_excel('/Users/talitacoll/The Bridge/MMP_Final/original_data.xlsx',usecols=cols)

In [9]:
df.shape

(284399, 9)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284399 entries, 0 to 284398
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                284399 non-null  object        
 1   doc_type          284399 non-null  object        
 2   transaction_date  284399 non-null  datetime64[ns]
 3   order_type        284399 non-null  object        
 4   order_date        284399 non-null  datetime64[ns]
 5   quantity          284399 non-null  int64         
 6   unit_cost         284399 non-null  float64       
 7   extended_cost     284399 non-null  float64       
 8   item_type         284399 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 19.5+ MB


**Starting the first clean up**

In [11]:
#Make the first cleaning to drop not relevants values
df = df.drop(df[(df['quantity'] == 0) | (df['unit_cost'] == 0.0)].index)

In [12]:
df.shape

(156366, 9)

*Detecting transaction where is not consider a demand*

In [13]:
df.order_type.unique()

array([' ', 'U1', 'E2', 'U2', 'UX', 'U3'], dtype=object)

In [14]:
#Only order type starting with U is consider usage
df = df[df.order_type.str.startswith('U')]

In [15]:
df.shape

(48810, 9)

*Detecting material that is not modules or spare*

In [16]:
df.item_type.unique()

array(['SPA', 'MOD', 'MAC', 'SOF'], dtype=object)

In [17]:
#Drop what is not modules or spares
df = df.drop(df[(df['item_type'] == 'MAC') | (df['item_type'] == 'SOF')].index)

In [18]:
df.shape

(47798, 9)

**How the document is based on existent data, but we want to replace by ficticious codes, we are starting the transforming data before start EDA**

In [19]:
#Split the dataframe in modules and spares
dfmod = df[df.item_type=='MOD']
dfspa = df[df.item_type=='SPA']

In [20]:
#This function is creating dataframes to supporting the replacement
def code_creat(a,b,c,d,e,f,g):
        a=[]
        for i in b:
            id = c + (d % random.randrange(e)).upper()
            a.append(id)
        f = pd.DataFrame(a,columns=[g]) 
        return f

In [21]:
#A dictionary with the features to make replacements in the dataframe
codes={'m':{'a':'mback', 'b':range(len(dfmod.id.value_counts())), 'c':'M', 'd':'%06x',
                'e':999999, 'f':'mdf', 'g':'cid'}, 
       's':{'a':'sback', 'b':range(len(dfspa.id.value_counts())), 'c':'S', 'd':'%08x', 
               'e':99999999, 'f':'sdf', 'g':'cid'},
      'p':{'a':'pback', 'b':range(45), 'c':'P', 'd':'%04x',
                'e':99999, 'f':'pdf', 'g':'id'}}

In [22]:
#Apply def in elements of dictionary
for k, v in codes.items():
    for i in k:
        exec(f'{k}df = code_creat(**v)')

In [23]:
#Check the def is working
print('Original and new df for spares len: ',len(dfspa.id.value_counts()),' and ',sdf.shape,'\n',
      'Original and new df for modules len: ',len(dfmod.id.value_counts()),' and ',mdf.shape,'\n',
      'and df for products: ', pdf.shape)

Original and new df for spares len:  1246  and  (1246, 1) 
 Original and new df for modules len:  789  and  (789, 1) 
 and df for products:  (45, 1)


In [24]:
sdf.head()

,cid
0,S031B79EA
1,S030F5A8F
2,S0322692D
3,S04164E76
4,S04ECF605


In [25]:
mdf.head()

,cid
0,M0B5DB4
1,M076BF6
2,M01E6ED
3,M02E559
4,M0B2C6F


In [26]:
pdf.head()

,id
0,P12FF3
1,P5DF0
2,PE390
3,P2AB0
4,PB91E


In [27]:
#Definy spares group to EDA and model
component_group = ['fix','plate','eletronic','mecanic','other']
for i in sdf:
    sdf['component_group'] = random.choices(component_group,k=len(sdf))

In [28]:
components = ([mdf,sdf])

In [29]:
idf = pd.concat(components,axis=0,join="outer",ignore_index=False)

In [30]:
idf.fillna('module',inplace=True)

In [31]:
idf.sample(10)

,cid,component_group
738,S01C9CCE1,fix
229,S04912A7C,fix
80,S03241183,plate
491,M04AC04,module
367,M0C924E,module
862,S00DC8557,mecanic
1179,S01523BFD,eletronic
696,M0B9917,module
486,M047703,module
328,M050E99,module


In [32]:
#Asign a quantity units per usage
def upm(idf):
    if idf.component_group != 'fix':
        f = random.randint(1,2)
    else:
        f = random.randint(1,8)*2
    return (f)

In [33]:
idf['upm'] = sdf.apply(upm,axis=1)

In [34]:
idf.sample(3)

,cid,component_group,upm
154,S04D6F70A,eletronic,1
1147,S010E4235,other,1
166,M057899,module,2


**CREATING CUSTOMER INSTALLED BASE**

*This part consist in create customer backlog to assign a local for each product serial number*

In [35]:
#Function to create sequential IDs
def id_create(a,b,c):
    b = []
    for i in a:
        id = c + '%.7d' % i
        b.append(id)
    return b

In [36]:
#Dictionary with serial number and customers features to create IDs
backlog={'bcustomers':{'a':range(1,200+1),'b':'bcustomers','c':'CS'},
         'serial_pack':{'a':range(1,12000+1),'b':'serial_pack','c':'SN'}}

In [37]:
#Execute backlog dictionary to create IDs
for k, v in backlog.items():
    for i in k:
        exec(f'{k} = id_create(**v)')

In [38]:
print(len(bcustomers),len(serial_pack))

200 12000


In [39]:
#Use the Fake Library we will create names and address to include in our customer backlog
fake = Faker('es_ES')
namelist = []
addresslist = []
for i in range(len(bcustomers)):
    namelist.append(fake.name())
    addresslist.append(fake.address())
    
cdf = pd.DataFrame(bcustomers,columns=['customer_id'])
k=len(bcustomers)
fake = Faker('es_ES')
for i in cdf.customer_id:
    cdf['name'] = random.choices(namelist,k=k)
    cdf['address']= random.choices(addresslist,k=k)

cdf.head(2)

,customer_id,name,address
0,CS0000001,Ángeles Ámbar Marco Mendoza,"Pasaje de Rita Moll 95\nSegovia, 11224"
1,CS0000002,Ernesto Alcázar Zamora,Paseo de Etelvina Mendoza 993 Puerta 3 \nCuenc...


In [40]:
#Fiting the address in the best way
cdf[['address','zone']] = cdf['address'].str.split(pat="\n",n=-1,expand=True)
cdf[['zone','cp']] = cdf['zone'].str.split(pat=",",n=-1,expand=True)

In [41]:
cdf.head()

,customer_id,name,address,zone,cp
0,CS0000001,Ángeles Ámbar Marco Mendoza,Pasaje de Rita Moll 95,Segovia,11224
1,CS0000002,Ernesto Alcázar Zamora,Paseo de Etelvina Mendoza 993 Puerta 3,Cuenca,25092
2,CS0000003,Manolo Cortes Boix,Plaza de Adrián Nogueira 41 Puerta 1,Cuenca,94571
3,CS0000004,Aura Canales Serna,Vial Marisol Amat 39 Puerta 0,Castellón,35790
4,CS0000005,Atilio Dalmau Alemany,Plaza Prudencio Torrecilla 14 Puerta 6,Alicante,30535


In [42]:
#around 4 minutes to run this block
#Creating the installed base in our customers
ibdf=pd.DataFrame(serial_pack,columns=['serial_id'])
for i in serial_pack:
  ibdf['product_id'] = random.choices(pdf.id,k=len(serial_pack))
  ibdf['customer_id'] = random.choices(bcustomers,k=len(serial_pack))

In [43]:
#Defyning the date when the machine has been installed in the customer
date1 = '2013-05-03'
date2 = '2017-05-10'
start = dt.datetime.strptime(date1, '%Y-%m-%d')
end = dt.datetime.strptime(date2, '%Y-%m-%d')
ibdf['install_date'] = random.choices((pd.date_range(date1,date2)),k=len(serial_pack))
ibdf.sample(2)

,serial_id,product_id,customer_id,install_date
3095,SN0003096,P1335E,CS0000121,2015-09-22
8679,SN0008680,P2423,CS0000146,2014-10-23


In [95]:
ibdf.sample(10)

,serial_id,product_id,customer_id,install_date
11961,SN0011962,PF501,CS0000035,2015-06-07
685,SN0000686,P121F,CS0000054,2014-05-10
5234,SN0005235,PE390,CS0000183,2013-12-04
3981,SN0003982,PFE6E,CS0000120,2014-01-30
11736,SN0011737,PB91E,CS0000001,2015-07-20
4502,SN0004503,P75B8,CS0000016,2014-03-13
2152,SN0002153,PFE6E,CS0000159,2016-11-08
5535,SN0005536,P152E9,CS0000004,2014-10-30
9744,SN0009745,P14627,CS0000149,2014-06-04
7412,SN0007413,PFE6E,CS0000116,2016-01-03


In [44]:
#To replaces IDs with fictial ID and keep ids quantity in the same proporcion, we will create a a new df with both columns and make a lookup
spa1 = pd.DataFrame(dfspa.id.drop_duplicates()).reset_index(drop=True)
spa2 = sdf.cid
spa = pd.concat([spa1.reset_index(drop=True),spa2.reset_index(drop=True)],axis=1)

In [45]:
spa.shape

(1246, 2)

In [46]:
dfspa.id = dfspa.id.map(spa.set_index('id')['cid'])

/Users/talitacoll/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [47]:
dfspa.sample(5)

,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type
218873,S057AF4F3,D1,2021-01-19,U1,2021-01-13,1,1.24,1.24,SPA
52007,S02FB42EB,D1,2018-03-09,U1,2018-02-22,1,144.45,144.45,SPA
61961,S05B86751,D1,2018-06-16,U1,2018-06-09,1,699.21,699.21,SPA
147623,S01419C73,D1,2019-12-05,U1,2019-12-02,3,80.87,242.61,SPA
138446,S04FBE91A,D1,2019-09-30,U1,2019-09-05,1,13.43,13.43,SPA


In [48]:
mod1 = pd.DataFrame(dfmod.id.drop_duplicates()).reset_index()
mod = pd.concat([mod1.id,mdf.cid],axis=1)

In [49]:
dfmod.id = dfmod.id.map(mod.set_index('id')['cid'])

/Users/talitacoll/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [50]:
dfmod.sample(5)

,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type
75712,M0718B8,D1,2018-07-22,U1,2018-07-12,1,58.0000,58.00,MOD
103395,M0203DB,D1,2019-02-16,U1,2019-02-06,1,60.3600,60.36,MOD
110413,M0E472F,D1,2019-04-09,U1,2019-03-12,1,832.1300,832.13,MOD
168723,M08C25C,D1,2020-03-31,U1,2020-03-26,1,225.4200,225.42,MOD
259189,M06BCE6,D1,2021-07-12,U3,2021-07-07,1,307.7378,307.74,MOD


In [51]:
components = idf['cid'].tolist()

In [90]:
#Create the Bill of Material for products using modules and spares as components
bom = {}
for i in pdf.id:
    for j in idf.cid:
        bom[i] = random.choices(components,k=random.randint(2,25))

In [53]:
df = pd.concat([dfspa,dfmod])

In [54]:
df.sort_index()

,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type
21525,S031B79EA,D1,2017-11-27,U1,2017-10-28,2,2.9800,5.96,SPA
21570,S030F5A8F,D1,2017-12-03,U1,2017-10-29,1,29.8400,29.84,SPA
21571,M0B5DB4,D1,2017-11-25,U1,2017-11-01,1,49.9400,49.94,MOD
21590,S0322692D,D1,2017-12-17,U1,2017-11-12,2,18.7300,37.46,SPA
21591,S04164E76,D1,2017-12-14,U1,2017-11-28,2,2.4300,4.86,SPA
...,...,...,...,...,...,...,...,...,...
284394,M05E910,D1,2017-12-15,U1,2017-12-08,1,24.6200,24.62,MOD
284395,M05E910,D1,2017-12-13,U1,2017-12-06,1,24.6200,24.62,MOD
284396,S0065CACD,D1,2021-12-23,U1,2021-11-20,1,51.0400,51.04,SPA
284397,M0E779F,D1,2021-12-17,U1,2021-11-19,1,440.0000,440.00,MOD


In [55]:
ibdf.head()

,serial_id,product_id,customer_id,install_date
0,SN0000001,P1643B,CS0000149,2015-01-21
1,SN0000002,P12FF3,CS0000098,2016-02-26
2,SN0000003,PA2E9,CS0000096,2017-04-16
3,SN0000004,PFE6E,CS0000153,2017-03-28
4,SN0000005,PC7D9,CS0000052,2014-04-15


In [56]:
# This code will definy each product belongs the record, according to BOM
for i in df.id:
    for k, v in bom.items():
        if i in v:
            product_id = pd.Series(random.choices(list(bom.keys()),k=len(df)),name='product_id')

In [57]:
df = pd.concat([df.reset_index(drop=False),product_id.reset_index(drop=True)],axis=1)

In [58]:
df.set_index('index',inplace=True)

In [59]:
df.head()

,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type,product_id
index,,,,,,,,,,
21525,S031B79EA,D1,2017-11-27,U1,2017-10-28,2,2.98,5.96,SPA,P146F0
21570,S030F5A8F,D1,2017-12-03,U1,2017-10-29,1,29.84,29.84,SPA,P146F0
21590,S0322692D,D1,2017-12-17,U1,2017-11-12,2,18.73,37.46,SPA,P1156
21591,S04164E76,D1,2017-12-14,U1,2017-11-28,2,2.43,4.86,SPA,P144B5
21592,S04ECF605,D1,2021-12-15,U1,2021-12-11,2,1.31,2.62,SPA,P144B5


In [60]:
serial = ibdf[['product_id','serial_id']]

In [61]:
serial = serial.groupby('product_id')['serial_id'].apply(list).to_dict()

In [63]:
len(serial.values())

45

In [64]:
# This code will definy each product belongs the record, according to BOM
for i in df.product_id:
    for k, v in serial.items():
        if i in k:
            serial_id = pd.Series(random.choices(list(serial.values())[1],k=len(df)),name='serial_id')

In [65]:
len(serial_id)

47798

In [66]:
df = pd.concat([df.reset_index(drop=False),serial_id.reset_index(drop=True)],axis=1)

In [67]:
df.head()

,index,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type,product_id,serial_id
0,21525,S031B79EA,D1,2017-11-27,U1,2017-10-28,2,2.98,5.96,SPA,P146F0,SN0003867
1,21570,S030F5A8F,D1,2017-12-03,U1,2017-10-29,1,29.84,29.84,SPA,P146F0,SN0004990
2,21590,S0322692D,D1,2017-12-17,U1,2017-11-12,2,18.73,37.46,SPA,P1156,SN0006929
3,21591,S04164E76,D1,2017-12-14,U1,2017-11-28,2,2.43,4.86,SPA,P144B5,SN0000202
4,21592,S04ECF605,D1,2021-12-15,U1,2021-12-11,2,1.31,2.62,SPA,P144B5,SN0000928


In [68]:
df = df.set_index('index')

In [69]:
df['install_date'] = df.serial_id.map(ibdf.set_index('serial_id')['install_date'])

In [70]:
#Calculate the gae of product in the moment of ticket
df['product_age']=df['order_date'] - df['install_date']
df['product_age']=df['product_age']/np.timedelta64(1,'D')

In [71]:
df.head()

,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type,product_id,serial_id,install_date,product_age
index,,,,,,,,,,,,,
21525,S031B79EA,D1,2017-11-27,U1,2017-10-28,2,2.98,5.96,SPA,P146F0,SN0003867,2014-06-04,1242.0
21570,S030F5A8F,D1,2017-12-03,U1,2017-10-29,1,29.84,29.84,SPA,P146F0,SN0004990,2014-08-03,1183.0
21590,S0322692D,D1,2017-12-17,U1,2017-11-12,2,18.73,37.46,SPA,P1156,SN0006929,2017-01-31,285.0
21591,S04164E76,D1,2017-12-14,U1,2017-11-28,2,2.43,4.86,SPA,P144B5,SN0000202,2015-04-07,966.0
21592,S04ECF605,D1,2021-12-15,U1,2021-12-11,2,1.31,2.62,SPA,P144B5,SN0000928,2016-01-10,2162.0


In [72]:
#Separate year and month from transaction date for EDA
df['year'] = pd.DatetimeIndex(df['order_date']).year
df['month'] = pd.DatetimeIndex(df['order_date']).month

In [73]:
df['leadtime']=df['transaction_date'] - df['order_date']
df['leadtime']=df['leadtime']/np.timedelta64(1,'D')

In [74]:
df = df.drop(df[(df['year'] == 2017)].index)

In [75]:
df = df.drop(df[(df['year'] == 2022)].index)

In [76]:
df.head()

,id,doc_type,transaction_date,order_type,order_date,quantity,unit_cost,extended_cost,item_type,product_id,serial_id,install_date,product_age,year,month,leadtime
index,,,,,,,,,,,,,,,,
21592,S04ECF605,D1,2021-12-15,U1,2021-12-11,2,1.31,2.62,SPA,P144B5,SN0000928,2016-01-10,2162.0,2021,12,4.0
21593,S03D2C47C,D1,2021-12-13,U1,2021-12-02,1,2.99,2.99,SPA,P16657,SN0010734,2015-10-19,2236.0,2021,12,11.0
21597,S03373322,D1,2021-12-12,U1,2021-12-08,4,3.29,13.16,SPA,P16657,SN0003017,2016-08-31,1925.0,2021,12,4.0
22017,S038C0C8E,D1,2021-12-14,U1,2021-12-05,1,29.33,29.33,SPA,PCBE1,SN0001489,2015-12-06,2191.0,2021,12,9.0
22041,S01FEA058,D1,2021-12-15,U1,2021-12-03,2,92.48,184.96,SPA,P52F1,SN0003013,2016-01-27,2137.0,2021,12,12.0


In [86]:
#With the dataset cleaned up, we are exporting to excel to have better data to explore and model
data = df.to_excel('/Users/talitacoll/The Bridge/MMP/data.xlsx')

In [87]:
inst_base = ibdf.to_excel('/Users/talitacoll/The Bridge/MMP/inst_base.xlsx')

**At this point the Data wrangling has been conclude and we can start EDA.**